# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [ ]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [ ]:
!pip install yelpapi --quiet

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [ ]:
# create a relative filepath
# import json
# import os

relative_path = os.path.join('.secret', 'yelp_api.json')

In [ ]:
# Load API Credentials
with open(relative_path) as file:
    login = json.load(file)

print(login.keys())

In [ ]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)
yelp_api

### Define Search Terms and File Paths

In [ ]:
# set our API call parameters and filename before the first call
LOCATION = 'Greenville, SC'
TERM = 'Sushi'

# filename for saving the data
# json_filename = 'yelp_search_resuls.json'

In [ ]:
## Specify fodler for saving data
# folder_path = r"D:\Documents\GitHub\data-enrichment-wk14-activity-mapping-yelp-api-results"

# Specifying JSON_FILE filename (can include a folder) => combine the folder path and filename
JSON_FILE = f'/Data/results_SC_Sushi.json'

print(f'data will be saved to: {JSON_FILE}')

### Check if Json File exists and Create it if it doesn't

In [ ]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)

## If it does not exist:
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder) > 0:
        # create the folder
        os.makedirs(folder, exist_ok = True)
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    ## save the first page of results
    with open(JSON_FILE, 'w') as file:
        json.dump([], file)
        
## If it exists, inform user
else:
    print(f'[i] {JSON_FILE} already exits.')

### Load JSON FIle and account for previous results

In [ ]:
## Load previous results and use len of results for offset
with open(JSON_FILE, 'r') as file:
    previous_results = json.load(file)
    
## set offset based on previous results
n_results = len(previous_results)

print(f'- {n_results} previous results found.')

### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [ ]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location = LOCATION,
                                term = TERM,
                                offset = n_results)

results.keys()

In [ ]:
## How many results total?
total_results = results['total']

total_results

- Where is the actual data we want to save?

In [ ]:
business_data = results['businesses']

# specify the filename where you want to save the data
json_file_path = JSON_FILE

# save the business data to a JSON file
with open(json_file_path, 'w') as file:
    json.dump(business_data, file, indent = 4)

In [ ]:
## How many did we get the details for?
results_per_page = len(business_data)
print(f'number of results retrieved per page', results_per_page)

- Calculate how many pages of results needed to cover the total_results

In [ ]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil(total_results / results_per_page)

print(f'Total number of pages: {n_pages}')

In [ ]:
for i in tqdm_notebook(range(1, n_pages + 1)):

    ## The block of code we want to TRY to run
    try:        
        # Introduce a short delay to respect API rate limits
        time.sleep(0.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as file:
            previous_results = json.load(file)
        
        ## Save number of results to use as offset
        n_results = len(previous_results)
        
        ## Use n_results as the OFFSET 
        results = yelp_api.search_query(location=LOCATION,
                                        term=TERM,
                                        offset=n_results)

        ## Append new results and save to file
        previous_results.extend(results['businesses'])
        with open(JSON_FILE, 'w') as file:
            json.dump(previous_results, file)
            
    ## What to do if we get an error/exception.
    except Exception as e:
        # check if we are at rate limit
        if 'Too Many Requests for url' in str(e):
            print('Rate limit exceeded. Stop data collection.')
            break
        else:
            print(f'an error occured {e}')
            # optionally handle error differently
            continue


## Open the Final JSON File with Pandas

In [ ]:
df = pd.read_json(JSON_FILE)
display(df.head(), df.tail())

In [ ]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

In [ ]:
# specify directory and filename
directory = 'Data'
filename = 'final_results_SC_Sushi'
path = os.path.join(directory, filename)

# ensure that the 'Data' directory exists
os.makedirs(directory, exist_ok = True)

## Save it as a compressed csv (to save space)
df.to_csv(path, compression = 'gzip', index = False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [ ]:
# # compare filesize with os module's os.path.getsize
# size_json = os.path.getsize(JSON_FILE)
# size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

# print(f'JSON FILE: {size_json:,} Bytes')
# print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

# print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

# Define the paths to the JSON and CSV.GZ files directly
json_file_path = 'Data/final_results_SC_Sushi.json'  # Update this path as necessary
csv_gz_file_path = 'Data/final_results_SC_Sushi.csv.gz'  # Update this path as necessary

# Check if both files exist
if os.path.exists(json_file_path) and os.path.exists(csv_gz_file_path):
    # Get the sizes of the files
    size_json = os.path.getsize(json_file_path)
    size_csv_gz = os.path.getsize(csv_gz_file_path)

    # Print file sizes
    print(f'JSON FILE: {size_json:,} Bytes')
    print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

    # Compare the sizes and avoid division by zero
    if size_csv_gz > 0:
        compression_ratio = size_json / size_csv_gz
        print(f'The csv.gz file is {compression_ratio:.2f} times smaller than the JSON file.')
    else:
        print("CSV.GZ file size is 0, cannot compare sizes.")
else:
    print("One or both files do not exist, check file paths.")


## Next Class: Processing the Results and Mapping 